In [1]:
# Dependencies
import numpy as np
import pandas as pd
import gmaps
import requests
import json
import pprint

#gmaps API Keys
from config import (gkey)

In [2]:
#Read Traffic Volume data
victrafficvol_df = pd.read_csv("Resources/Traffic_Volume.csv")
victrafficvol_df.head()

,OBJECTID_1,OBJECTID,TIS_ID,HMGNS_FLOW_ID,HMGNS_LNK_ID,HMGNS_LNK_DESC,LGA_SHORT_NM,RGN_LONG_NM,ROAD_NBR,DECLARED_ROAD_NM,...,TWO_WAY_AADT_TRUCKS,ALLVEH_AMPEAK_AADT,ALLVEH_PMPEAK_AADT,GROWTH_RATE,CI,AM_PEAK_SPEED,OFF_PEAK_SPEED,PM_PEAK_SPEED,YR,LABEL
0,1,743,14915,14915,2006,MARYSVILLE-WOODS POINT ROAD btwn LAKE MOUNTAI...,YARRA RANGES,METROPOLITAN SOUTH EAST REGION,4961,MARYSVILLE-WOODS POINT ROAD,...,0.0,NaN,NaN,0.013,0.005,NaN,NaN,NaN,2020,24* (13% 3*) EAST BOUND
1,2,650,14140,14140,8786,STEELS CREEK ROAD btwn WILLOWBEND DRIVE & ELT...,YARRA RANGES,METROPOLITAN SOUTH EAST REGION,9999,Not Applicable,...,40.0,NaN,NaN,0.019,0.002,NaN,NaN,NaN,2020,373* (6% 22*) NORTH BOUND
2,3,701,12113,12113,6035,LATROBE ROAD btwn TANJIL EAST ROAD & GORDON S...,LATROBE,EASTERN REGION,5911,MORWELL-YALLOURN NORTH ROAD,...,160.0,NaN,NaN,0.015,0.009,NaN,NaN,NaN,2020,"1,100* (6% 61*) NORTH BOUND"
3,4,702,12897,12897,7079,CASTERTON ROAD btwn GLENELG HIGHWAY & COLERAI...,SOUTHERN GRAMPIANS,SOUTH WESTERN REGION,2670,GLENELG HIGHWAY,...,340.0,NaN,NaN,0.020,0.001,NaN,NaN,NaN,2020,801* (21% 165*) WEST BOUND
4,5,703,9893,9893,3475,HUTTON ROAD btwn CHAPEL ROAD & GREENS ROAD,DANDENONG,METROPOLITAN SOUTH EAST REGION,5168,BRAESIDE-DANDENONG ROAD,...,1500.0,1000.0,1100.0,0.003,0.002,NaN,NaN,NaN,2020,"12,000 (6% 744*) WEST BOUND"


In [3]:
#Read homogenous traffic flow data
trafficflow_df = pd.read_csv("Resources/homogenous_traffic_flow.csv")
trafficflow_df.head()

,OBJECTID,TIS_ID,TIS_TYPE,TIS_ROUTE_,TIS_ROUTE1,TIS_DESC,MIDPNT_LAT,MIDPNT_LON,DT_MODIFIE,DT_CREATED,HMGNS_FLOW,HMGNS_FL_1,HMGNS_FL_2,ROAD_NBR,DECLARED_R,LRS_ROAD_D,LRS_ROAD_1
0,15496,1169,HFLOW_2,221,1,EAGLEHAWK ROAD S Bd btwn LONG GULLY-WHITE HILL...,-36.743736,144.258338,1404482718000,1404482718000,1169,S,SOUTH BOUND,2630,LODDON VALLEY HIGHWAY,1,REVERSE
1,15497,979,HFLOW_2,845,1,ST KILDA RD S BD BTWN INKERMAN ST AND CARLISLE...,-37.866362,144.986438,1404482726000,1404482726000,979,S,SOUTH BOUND,2660,NEPEAN HIGHWAY,0,FORWARD
2,15498,981,HFLOW_2,845,1,NEPEAN HWY S BD BTWN NORTH RD AND BAY ST || Or...,-37.903167,145.007445,1404482734000,1404482734000,981,S,SOUTH BOUND,2660,NEPEAN HIGHWAY,0,FORWARD
3,15499,1525,HFLOW_2,845,1,NEPEAN HWY S BD BTWN STATION ST AND WICKHAM RD...,-37.940626,145.040047,1404482742000,1404482742000,1525,S,SOUTH BOUND,2660,NEPEAN HIGHWAY,0,FORWARD
4,15500,796,HFLOW_2,845,1,NEPEAN HWY E BD BTWN WARRIGAL RD AND LOWER DAN...,-37.982354,145.072737,1404482750000,1404482750000,796,E,EAST BOUND,2660,NEPEAN HIGHWAY,0,FORWARD


In [4]:
#Remove unneccesary columns from both tables

victrafficvol_df = victrafficvol_df[["TIS_ID", "HMGNS_LNK_DESC", "LGA_SHORT_NM", "ALLVEHS_AADT",
                                    "TRUCKS_AADT","PER_TRUCKS_AADT"]]

trafficflow_df= trafficflow_df[["TIS_ID", "MIDPNT_LAT", "MIDPNT_LON"]]

In [5]:
#Merge both tables on ROAD_NBR
victrafficvol_merged_df = pd.merge(victrafficvol_df, trafficflow_df, how="left", on="TIS_ID")
victrafficvol_merged_df.isnull().sum()

TIS_ID               0
HMGNS_LNK_DESC       8
LGA_SHORT_NM         0
ALLVEHS_AADT         0
TRUCKS_AADT        698
PER_TRUCKS_AADT    698
MIDPNT_LAT          12
MIDPNT_LON          12
dtype: int64

In [6]:
#Remove rows with no lat or long
victrafficvol_nonulls = victrafficvol_merged_df.dropna(axis=0, subset=['MIDPNT_LAT'])
victrafficvol_nonulls.isnull().sum()

TIS_ID               0
HMGNS_LNK_DESC       8
LGA_SHORT_NM         0
ALLVEHS_AADT         0
TRUCKS_AADT        691
PER_TRUCKS_AADT    691
MIDPNT_LAT           0
MIDPNT_LON           0
dtype: int64

In [7]:
#Rename Coloumns
victrafficvol_cleaned_df = victrafficvol_nonulls.rename(columns={"HMGNS_LNK_DESC": "LOCATION_LINK_DESC", "LGA_SHORT_NM": "LGA", 
                                                                 "ALLVEHS_AADT": "ALL_VEHICLE_DAILY_VOLUME_AVG", "TRUCKS_AADT": "TRUCKS_DAILY_VOLUME_AVG",
                                                                "PER_TRUCKS_AADT": "DAILY_AVG_%_OF_VOLUME_OF_TRUCKS", "MIDPNT_LAT": "LATITUDE", 
                                                                 "MIDPNT_LON" : "LONGITUDE"})

victrafficvol_cleaned_df.head()

,TIS_ID,LOCATION_LINK_DESC,LGA,ALL_VEHICLE_DAILY_VOLUME_AVG,TRUCKS_DAILY_VOLUME_AVG,DAILY_AVG_%_OF_VOLUME_OF_TRUCKS,LATITUDE,LONGITUDE
0,14915,MARYSVILLE-WOODS POINT ROAD btwn LAKE MOUNTAI...,YARRA RANGES,24,3.0,0.13,-37.559409,145.864626
1,14140,STEELS CREEK ROAD btwn WILLOWBEND DRIVE & ELT...,YARRA RANGES,373,22.0,0.07,-37.628415,145.373573
2,12113,LATROBE ROAD btwn TANJIL EAST ROAD & GORDON S...,LATROBE,1100,61.0,0.07,-38.195167,146.402951
3,12897,CASTERTON ROAD btwn GLENELG HIGHWAY & COLERAI...,SOUTHERN GRAMPIANS,801,165.0,0.25,-37.579418,141.564559
4,9893,HUTTON ROAD btwn CHAPEL ROAD & GREENS ROAD,DANDENONG,12000,744.0,0.07,-38.013205,145.165515


In [8]:
#Run loop through lat and long for each row to find suburb and postcode from maps api

for index, row in victrafficvol_cleaned_df.iterrows():
    lat = row["LATITUDE"]
    lng = row["LONGITUDE"]

    query_url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={gkey}"

    locations = requests.get(query_url).json()
    
    try:

        if len(locations["results"][0]["formatted_address"].split(",")) == 3:
            victrafficvol_cleaned_df.loc[index, "SUBURB"] = locations["results"][0]["formatted_address"].split(",")[1].split("VIC")[0]
        else:
            victrafficvol_cleaned_df.loc[index, "SUBURB"] = locations["results"][0]["formatted_address"].split(",")[2].split("VIC")[0]

        if len(locations["results"][0]["formatted_address"].split(",")) == 3:
            victrafficvol_cleaned_df.loc[index, "POSTCODE"] = locations["results"][0]["formatted_address"].split(",")[1].split("VIC")[1]
        else:
            victrafficvol_cleaned_df.loc[index, "POSTCODE"] = locations["results"][0]["formatted_address"].split(",")[2].split("VIC")[1]
        
    except (KeyError, IndexError):
         print(f"Missing field/result... skipping index number {index}.")

Missing field/result... skipping index number 129.
Missing field/result... skipping index number 955.
Missing field/result... skipping index number 1410.
Missing field/result... skipping index number 1935.
Missing field/result... skipping index number 2197.
Missing field/result... skipping index number 2360.
Missing field/result... skipping index number 2652.
Missing field/result... skipping index number 2671.
Missing field/result... skipping index number 2684.
Missing field/result... skipping index number 2818.
Missing field/result... skipping index number 3188.
Missing field/result... skipping index number 3283.
Missing field/result... skipping index number 3483.
Missing field/result... skipping index number 4120.
Missing field/result... skipping index number 4182.
Missing field/result... skipping index number 4429.
Missing field/result... skipping index number 4955.
Missing field/result... skipping index number 4976.
Missing field/result... skipping index number 5086.
Missing field/

'Marysville-Woods Point Rd, Cambarville VIC 3779, Australia'

In [9]:
victrafficvol_cleaned_df.isnull().sum()

TIS_ID                               0
LOCATION_LINK_DESC                   8
LGA                                  0
ALL_VEHICLE_DAILY_VOLUME_AVG         0
TRUCKS_DAILY_VOLUME_AVG            691
DAILY_AVG_%_OF_VOLUME_OF_TRUCKS    691
LATITUDE                             0
LONGITUDE                            0
dtype: int64

In [21]:
victrafficvol_cleaned_final_df = victrafficvol_cleaned_df[victrafficvol_cleaned_df['POSTCODE'].notna()]

In [24]:
victrafficvol_cleaned_final_df.isnull().sum()

TIS_ID                               0
LOCATION_LINK_DESC                   8
LGA                                  0
ALL_VEHICLE_DAILY_VOLUME_AVG         0
TRUCKS_DAILY_VOLUME_AVG            691
DAILY_AVG_%_OF_VOLUME_OF_TRUCKS    691
LATITUDE                             0
LONGITUDE                            0
SUBURB                               0
POSTCODE                             0
dtype: int64

In [25]:
victrafficvol_cleaned_final_df.to_csv('Export/Traffic_Volume.csv',index=False)

In [2]:
#Reread new Traffic Volume data (Don't have to rerun api)
victrafficvol_cleaned_final_df = pd.read_csv("Export/Traffic_Volume.csv")
victrafficvol_cleaned_final_df.head()

,TIS_ID,LOCATION_LINK_DESC,LGA,ALL_VEHICLE_DAILY_VOLUME_AVG,TRUCKS_DAILY_VOLUME_AVG,DAILY_AVG_%_OF_VOLUME_OF_TRUCKS,LATITUDE,LONGITUDE,SUBURB,POSTCODE
0,14915,MARYSVILLE-WOODS POINT ROAD btwn LAKE MOUNTAI...,YARRA RANGES,24,3.0,0.13,-37.559410,145.864626,Cambarville,3779
1,14140,STEELS CREEK ROAD btwn WILLOWBEND DRIVE & ELT...,YARRA RANGES,373,22.0,0.07,-37.628415,145.373573,Yarra Glen,3775
2,12113,LATROBE ROAD btwn TANJIL EAST ROAD & GORDON S...,LATROBE,1100,61.0,0.07,-38.195167,146.402951,Maryvale,3840
3,12897,CASTERTON ROAD btwn GLENELG HIGHWAY & COLERAI...,SOUTHERN GRAMPIANS,801,165.0,0.25,-37.579418,141.564559,Coleraine,3315
4,9893,HUTTON ROAD btwn CHAPEL ROAD & GREENS ROAD,DANDENONG,12000,744.0,0.07,-38.013205,145.165515,Keysborough,3173


In [8]:
#Find data on traffic volume grouped by postcode

postcode_allveh = victrafficvol_cleaned_final_df.groupby(["POSTCODE"])["ALL_VEHICLE_DAILY_VOLUME_AVG"].sum()
postcode_trucks = victrafficvol_cleaned_final_df.groupby(["POSTCODE"])["TRUCKS_DAILY_VOLUME_AVG"].sum()
postcode_truckspercent = victrafficvol_cleaned_final_df.groupby(["POSTCODE"])["DAILY_AVG_%_OF_VOLUME_OF_TRUCKS"].mean()

In [13]:
#Create dataframe with trafic volume grouped by postcode

trafficvol_postcode = pd.DataFrame({"ALL_VEHICLE_DAILY_VOLUME_AVG":postcode_allveh, "TRUCKS_DAILY_VOLUME_AVG": postcode_trucks, "DAILY_AVG_%_OF_VOLUME_OF_TRUCKS": round(postcode_truckspercent,3)}).reset_index()
trafficvol_postcode.head()

,POSTCODE,ALL_VEHICLE_DAILY_VOLUME_AVG,TRUCKS_DAILY_VOLUME_AVG,DAILY_AVG_%_OF_VOLUME_OF_TRUCKS
0,3000,1370400,66451.0,0.070
1,3002,590610,30120.0,0.062
2,3003,1442616,132243.0,0.095
3,3004,1461086,90237.0,0.064
4,3006,1340537,94574.0,0.071


In [14]:
trafficvol_postcode.to_csv('Export/Traffic_Volume_Postcode.csv',index=False)